In [ ]:
#| default_exp tui


# TUI - Terminal User Interface

Textual-based terminal user interface components for audio transcription.


In [ ]:
#| export
from time import monotonic
from enum import Enum, auto
import asyncio

from textual import on
from textual.app import App, ComposeResult
from textual.reactive import reactive
from textual.containers import Container, VerticalGroup, Grid, HorizontalGroup, CenterMiddle
from textual.widgets import Header, Footer, Static, Digits, Button, Label, TextArea, Log

from tui_writer.live import LiveTranscriber
from tui_writer.ai import has_session, start_session, apply_instruction, current_transcript, reset_session

In [ ]:
#| export

TEXTUAL_CSS = """
    Screen {
        background: $surface;
    }
    Header {
        background: $primary;
    }

    StatusCard {
        width: auto;
        row-span: 2;
        border: solid $primary;
        border-title-align: center;
        height: 100%;
        padding: 1 0;
    }

    InfoCard {
        width: auto;
        border: solid #6b7280;
        border-title-align: center;
        height: 100%;
        column-span: 3;
        padding: 1;
    }

    #transcript-field {
        width: auto;
        border: solid $warning;
        border-title-align: center;
        height: 100%;
        column-span: 3;
        padding: 1;
    }

    #top-cards {
        layout: grid;
        grid-size: 4 2;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
        margin: 1;
        height: 40vh;
    }

    #textfield {
        width: 100%;
        margin: 1;
        padding: 1;
        border: solid #8b5cf6;
        border-title-align: center;
        height: 50vh;
    }

    #status {
        content-align: center middle;
    }

    #status-time {
        border: double green;
        width: auto;
        padding: 0 1;
        margin: 1 0;
    }

    #media-controls {
        dock: bottom;
        align: center middle;
    }

    #media-controls Button {
        min-width: 20;
    }

    #pause,
    #resume,
    #stop {
        display: none;
    }

    .active #start {
        display: none;
    }
    .active #pause,
    .active #stop {
        display: block;
    }
    .active #status {
        color: $error;
    }

    .paused #start {
        display: none;
    }
    
    .paused #status {
        color: $warning;
    }

    .paused #resume,
    .paused #stop {
        display: block;
    }
"""

In [ ]:
#| export
class TimeDisplay(Digits): 
    """Custom time display widget""" 
    accumulated_time = 0 
    time_elapsed = reactive(0) 
    def on_mount(self): 
        self.update_timer = self.set_interval( 
            1 / 60, 
            self.update_time_elapsed, 
            pause=True 
        )
    
    def update_time_elapsed(self): 
        self.time_elapsed = self.accumulated_time + (
            monotonic() - self.start_time
        ) 
        
    def watch_time_elapsed(self): 
        time = self.time_elapsed 
        time, seconds = divmod(time, 60) 
        hours, minutes = divmod(time, 60) 
        time_string = f"{hours:02.0f}:{minutes:02.0f}:{seconds:05.2f}" 
        self.update(time_string) 
        
    def start(self): 
        self.start_time = monotonic() 
        self.update_timer.resume() 
        
    def pause(self): 
        self.accumulated_time = self.time_elapsed 
        self.update_timer.pause() 
        
    def stop(self): 
        self.start_time = monotonic() 
        self.accumulated_time = 0 
        self.time_elapsed = 0 
        self.update_timer.pause()

class StatusCard(Label):
    """Recording status card with controls"""
    status_text = reactive("● STANDBY")

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Recording Status"
        self._status_label: Static | None = None

    def start_transcript(self):
        self.add_class("active")
        self.status_text = "● RECORDING..."

    def pause_transcript(self):
        self.remove_class("active")
        self.add_class("paused")
        self.status_text = "⏸ PAUSED"

    def resume_transcript(self):
        self.remove_class("paused")
        self.add_class("active")
        self.status_text = "● RECORDING..."

    def stop_transcript(self):
        self.remove_class("paused")
        self.remove_class("active")
        self.status_text = "● STANDBY"
    
    def on_mount(self) -> None:
        self._status_label = self.query_one("#status", Static)

    def watch_status_text(self) -> None:
        if self.is_mounted and self._status_label:
            self._status_label.update(self.status_text)

    def compose(self) -> ComposeResult:
        yield Static(self.status_text, id="status")
        with CenterMiddle():
            yield TimeDisplay("0:00:00", id="status-time")
        with HorizontalGroup(id="media-controls"):
            yield Button.error("● REC <space>", id="start")
            yield Button.success("⏸ PAUSE <space>", id="pause")
            yield Button.warning("▶ RESUME <space>", id="resume")
            yield Button("◼ STOP", id="stop")
    
class InfoCard(Label):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Info"

    def compose(self) -> ComposeResult:
        yield Static("Model: Whisper-Tiny")
        yield Static("Microphone: Default System Mic")

## TranscriptionTUI

Main terminal user interface application for audio transcription.

The TUI uses Textual's [theme system](https://textual.textualize.io/guide/design/) with built-in color variables like `$primary`, `$error`, `$success`, etc. 

**Available built-in themes:**
- `textual-dark` (default)
- `textual-light`
- `nord`
- `gruvbox`
- `tokyo-night`
- `solarized-light`

To change the theme, modify the `self.theme` assignment in `__init__`.


In [ ]:
#| export
class RecordingState(Enum):
    IDLE = auto()
    RECORDING = auto()
    PAUSED = auto()

class TranscriptionTUI(App):

    TITLE = "TUI Writer"
    SUB_TITLE = "Design Template"
    CSS = TEXTUAL_CSS

    state: RecordingState = RecordingState.IDLE

    BINDINGS = [
        ("q", "quit", "Quit"),
        ("space", "toggle_recording", "Start/Pause/Resume"),
        ("l", "list_models", "List Models"),
        ("s", "stop_recording", "Stop"),
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._task: asyncio.Task | None = None
        self._transcriber: LiveTranscriber | None = None
        self.all_chunks: list[str] = []

    def compose(self) -> ComposeResult:
        header = Header(show_clock=True)
        header.tall = True

        self.transcript_field = Label("hrifherfeifiohrfhioerfihefherhifoiehfoih",id="transcript-field")


        textfield = Label(id="textfield")
        textfield.border_title = "Full Transcript"
        yield header

        with Container(id="top-cards"):
            yield StatusCard()
            yield InfoCard()  # Add an Info widget here
            yield self.transcript_field
    
        with textfield:
            yield Log(
                id="main-textarea",
            )
        yield Footer()

    # Action methods triggered by key bindings, refers to button-methods
    async def action_toggle_recording(self) -> None:
        if self.state is RecordingState.IDLE:
            await self._start()
        elif self.state is RecordingState.RECORDING:
            await self._pause()
        elif self.state is RecordingState.PAUSED:
            await self._resume()
    
    async def action_stop_recording(self) -> None:
        await self._stop()
    
    @on(Button.Pressed, "#start")
    async def _on_start(self, _):
        await self._start()

    @on(Button.Pressed, "#pause")
    async def _on_pause(self, _):
        await self._pause()

    @on(Button.Pressed, "#resume")
    async def _on_resume(self, _):
        await self._resume()

    @on(Button.Pressed, "#stop")
    async def _on_stop(self, _):
        await self._stop()


    async def _start(self) -> None:
        if self.state is not RecordingState.IDLE:
            return
        self._transcriber = LiveTranscriber(
            model_id="tiny", language="en",
            on_transcript=self.on_transcript_chunk,
            vad_threshold=0.5, min_speech_duration_ms=250, min_silence_duration_ms=500,
        )
        self._task = asyncio.create_task(self._transcriber.start())
        self.status_card.start_transcript()
        self.time_display.start()
        self.state = RecordingState.RECORDING

    async def _pause(self) -> None:
        if self.state is not RecordingState.RECORDING:
            return
        try:
            if self._transcriber:
                self._transcriber.stop()
            if self._task:
                await self._task
            if self._transcriber:
                await self._transcriber.flush()
        finally:
            self._task = None
            self._transcriber = None
            self.time_display.pause()
            self.status_card.pause_transcript()
            self.state = RecordingState.PAUSED

    async def _resume(self) -> None:
        if self.state is not RecordingState.PAUSED:
            return
        self._transcriber = LiveTranscriber(
            model_id="tiny",
            language="en",
            on_transcript=self.on_transcript_chunk,
            vad_threshold=0.5,
            min_speech_duration_ms=250,
            min_silence_duration_ms=500,
        )
        self._task = asyncio.create_task(self._transcriber.start())
        self.time_display.start()
        self.status_card.resume_transcript()
        self.state = RecordingState.RECORDING

    async def _stop(self) -> None:
        if self.state is RecordingState.IDLE:
            return
        try:
            if self._transcriber:
                self._transcriber.stop()
            if self._task:
                await self._task
            if self._transcriber:
                await self._transcriber.flush()
        finally:
            self._transcriber = None
            self._task = None
            self.time_display.stop()
            self.status_card.stop_transcript()
            self.state = RecordingState.IDLE

    def on_transcript_chunk(self, text: str) -> None:
        text = (text or "").strip()
        if not text or self.state is RecordingState.IDLE:
            return
        self.all_chunks.append(text + "\n")
        self.transcript_block.update(text)
        self.main_textarea.write_line(text)

    def on_mount(self) -> None:
        self.status_card: StatusCard = self.query_one(StatusCard)
        self.time_display: TimeDisplay = self.query_one(TimeDisplay)
        self.transcript_block: Label = self.query_one("#transcript-field", Label)
        self.main_textarea: Log = self.query_one("#main-textarea", Log)
        self.transcript_field.border_title = "Last block of speech"

    async def on_unmount(self) -> None:
        await self._stop()